Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [2]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [3]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

,player,starter,percentage
0,Falcone,1,90.0
1,Gendrey,1,90.0
2,Baschirotto,1,90.0
3,Pongracic,1,80.0
4,Gallo,1,60.0
...,...,...,...
462,Oristanio,0,50.0
463,Jankto,0,15.0
464,Mancosu,0,10.0
465,Pavoletti,0,55.0


In [4]:
probables = probables.set_index('player')

In [5]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

,player1,player2,percentage
0,Gallo,Dorgu,60.0
1,Blin,Gonzalez J.,60.0
2,Politano,Lindstrom,60.0
3,Zambo Anguissa,Cajuste,60.0
4,Zielinski,Raspadori,55.0
5,Kvaratskhelia,Elmas,65.0
6,Mario Rui,Olivera,55.0
7,Calabria,Florenzi,55.0
8,Pulisic,Chukwueze,55.0
9,Reijnders,Musah,65.0


In [6]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

,starter,percentage
player,,
Falcone,1,90.0
Gendrey,1,90.0
Baschirotto,1,90.0
Pongracic,1,80.0
Gallo,0.6,60.0
...,...,...
Oristanio,0,50.0
Jankto,0,15.0
Mancosu,0,10.0


In [7]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   